# Notes
These are just notes I used for constructing the TFP joints

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util
tfp.__version__

'0.11.0'

In [2]:
import maxentep

In [19]:
i =  tf.keras.Input((1,))
logits = np.array(np.random.normal(size=10)).astype(np.float32)
l = maxentep.categorical_normal_layer(i,logits, 0.2, 0.01, 'cn')

In [23]:
model = tf.keras.Model(inputs=i, outputs=l)
j = model(tf.constant([1.]))

In [26]:
j.sample(5)

[<tf.Tensor: shape=(5, 1, 10), dtype=float32, numpy=
 array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]],
 
        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[0.19137411],
        [0.20370506],
        [0.21425095],
        [0.20338118],
        [0.19698074]], dtype=float32)>,
 <tf.Tensor: shape=(5, 5, 10), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.19137411, 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.20370506, 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.21425095, 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ],
        

In [5]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
print(tf.where(bs > 0)[:,1:])


tf.Tensor(
[[0 1]
 [1 2]
 [2 3]
 [3 3]
 [0 1]
 [1 2]
 [2 3]
 [3 3]], shape=(8, 2), dtype=int64)


In [6]:
j = maxentep.recip_norm_mat_dist(start, var)
print(j)
j.sample()

tfp.distributions.JointDistributionSequentialAutoBatched("JointDistributionSequentialAutoBatched", batch_shape=[], event_shape=[[4], [4], [4, 4]], dtype=[float64, float64, float64])


[<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.08224972, 0.33585882, 0.61293948, 0.70484653])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.91775028, 0.66414118, 0.38706052, 0.29515347])>,
 <tf.Tensor: shape=(4, 4), dtype=float64, numpy=
 array([[0.91775028, 0.08224972, 0.        , 0.        ],
        [0.        , 0.66414118, 0.33585882, 0.        ],
        [0.        , 0.        , 0.38706052, 0.61293948],
        [0.        , 0.        , 0.        , 1.        ]])>]

In [7]:
j.log_prob(j.sample())

<tf.Tensor: shape=(), dtype=float64, numpy=3.1583856434440656>

In [8]:
x = j.log_prob(j.sample(5))

In [9]:
print(j.sample(4))
#j.log_prob(j.sample())

[<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.1795317 , 0.58183253, 0.91377226, 0.67009096],
       [0.07393932, 0.90763678, 0.43597695, 0.79950142],
       [0.12180175, 0.88102409, 0.38727057, 0.79872368],
       [0.34486626, 0.22472246, 0.27991893, 0.86118619]])>, <tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.8204683 , 0.41816747, 0.08622774, 0.32990904],
       [0.92606068, 0.09236322, 0.56402305, 0.20049858],
       [0.87819825, 0.11897591, 0.61272943, 0.20127632],
       [0.65513374, 0.77527754, 0.72008107, 0.13881381]])>, <tf.Tensor: shape=(4, 4, 4), dtype=float64, numpy=
array([[[0.8204683 , 0.1795317 , 0.        , 0.        ],
        [0.        , 0.41816747, 0.58183253, 0.        ],
        [0.        , 0.        , 0.08622774, 0.91377226],
        [0.        , 0.        , 0.        , 1.        ]],

       [[0.92606068, 0.07393932, 0.        , 0.        ],
        [0.        , 0.09236322, 0.90763678, 0.        ],
        [0.        , 0.        , 0.56402

In [10]:
i =  tf.keras.Input((1,))
combined = np.stack((start, var))
print(combined.shape)
x  = maxentep.TrainableInputLayer(combined, constraint=maxentep.PositiveMaskedConstraint(combined > 0))(i)

(2, 4, 4)


In [11]:
m = tf.keras.Model(inputs=i, outputs=x)
t = m(tf.constant([1., 1.]))
t

<tf.Tensor: shape=(2, 2, 4, 4), dtype=float32, numpy=
array([[[[ 0. , 10. ,  0. ,  0. ],
         [ 0. ,  0. ,  3. ,  0. ],
         [ 0. ,  0. ,  0. ,  2. ],
         [ 0. ,  0. ,  0. ,  1. ]],

        [[ 0. ,  5. ,  0. ,  0. ],
         [ 0. ,  0. ,  1.5,  0. ],
         [ 0. ,  0. ,  0. ,  1. ],
         [ 0. ,  0. ,  0. ,  0.5]]],


       [[[ 0. , 10. ,  0. ,  0. ],
         [ 0. ,  0. ,  3. ,  0. ],
         [ 0. ,  0. ,  0. ,  2. ],
         [ 0. ,  0. ,  0. ,  1. ]],

        [[ 0. ,  5. ,  0. ,  0. ],
         [ 0. ,  0. ,  1.5,  0. ],
         [ 0. ,  0. ,  0. ,  1. ],
         [ 0. ,  0. ,  0. ,  0.5]]]], dtype=float32)>

In [12]:
t[0,0], t[0,1]

(<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[ 0., 10.,  0.,  0.],
        [ 0.,  0.,  3.,  0.],
        [ 0.,  0.,  0.,  2.],
        [ 0.,  0.,  0.,  1.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[0. , 5. , 0. , 0. ],
        [0. , 0. , 1.5, 0. ],
        [0. , 0. , 0. , 1. ],
        [0. , 0. , 0. , 0.5]], dtype=float32)>)

In [13]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_mat_layer(i, start, var, 'test')

In [14]:
m = tf.keras.Model(inputs=i, outputs=l)

In [15]:
m(tf.constant([1.]))

<tfp.distributions.JointDistributionSequentialAutoBatched 'functional_5_test_dist_JointDistributionSequentialAutoBatched' batch_shape=[] event_shape=[[4], [4], [4, 4]] dtype=[float32, float32, float32]>

In [16]:
logits = np.array(np.random.normal(size=10)).astype(np.float32)
d = tfd.Multinomial(1, logits)
d

<tfp.distributions.Multinomial 'Multinomial' batch_shape=[] event_shape=[10] dtype=float32>